In [1]:
!pip install flask transformers requests

In [2]:
%%writefile flask_chatbot_app.py
from flask import Flask, jsonify
import requests
from transformers import pipeline

app = Flask(__name__)

chatbot_model = pipeline("text-generation", model="microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)
conversation_history = []

@app.route('/chatbot', methods=['POST'])
def chatbot():
  userInput = requests.json.get('message')
  if not userInput:
    return jsonify({'response':'Please provide a message'}), 400
  response = generate_response(userInput)
  return jsonify({'response':response})


def generate_response(userInput):
  # generate a response using the Hugging Face model
  message = {
      {"role":"user", "content":userInput},
  }

  result = chatbot_model(conversation_history, max_new_tokens=250, num_return_sequences=1)
  conversation_history.append(
      {"role":"assistant", "content":result[0]['generated_text'][1]['content']},
  )
  return result[0]['generated_text'][1]['content']


if __name__ == '__main__':
  app.run(debug=True, port=5000, host='0.0.0.0')

Writing flask_chatbot_app.py


In [3]:
import subprocess

subprocess.run(['pkill', '-f', 'flask_chatbot_app.py'])

CompletedProcess(args=['pkill', '-f', 'flask_chatbot_app.py'], returncode=1)

In [4]:
!nohup python flask_chatbot_app.py &

nohup: appending output to 'nohup.out'


In [22]:
!sudo lsof -i -P -n | grep LISTEN

node        7 root   21u  IPv6  22134      0t0  TCP *:8080 (LISTEN)
kernel_ma  19 root    3u  IPv4  21913      0t0  TCP 172.28.0.12:6000 (LISTEN)
colab-fil  65 root    3u  IPv4  22144      0t0  TCP 127.0.0.1:3453 (LISTEN)
jupyter-n 114 root    7u  IPv4  22508      0t0  TCP 172.28.0.12:9000 (LISTEN)
python3   291 root   21u  IPv4  28464      0t0  TCP 127.0.0.1:40217 (LISTEN)
python3   326 root    3u  IPv4  31140      0t0  TCP 127.0.0.1:41611 (LISTEN)
python3   326 root    4u  IPv4  31141      0t0  TCP 127.0.0.1:42385 (LISTEN)


In [ ]:
# 127.0.0.1:42385

In [18]:
import requests

url = 'https://127.0.0.1:5000/chatbot'

response = requests.post(url, json={"message":"hello, how are you today"})
print(response.json())

ConnectionError: HTTPSConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /chatbot (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7a336d5c7be0>: Failed to establish a new connection: [Errno 111] Connection refused'))